In [1]:
import pandas as pd
import numpy as np

from ortools.linear_solver import pywraplp

import random

pd.set_option('display.max_rows', 50)


from functions import get_tickers_stocks, get_close_prices, double_listed_stocks, sharpe_ratio_calculation, generate_rand_portfolios, select_top_five

load c:\Users\Kirill\Documents\Projects\thesis\cleanenv\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\Kirill\Documents\Projects\thesis\cleanenv\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\Kirill\Documents\Projects\thesis\cleanenv\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\Kirill\Documents\Projects\thesis\cleanenv\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\Kirill\Documents\Projects\thesis\cleanenv\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\Kirill\Documents\Projects\thesis\cleanenv\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\Kirill\Documents\Projects\thesis\cleanenv\Lib\site-packages\ortools\.libs\ortools.dll...


In [2]:
#GET THE STOCKS

us_exchanges = ['NMS', 'NYQ', 'NGM']
eu_exchanges = ['PAR', 'FRA', 'LSE', 'AMS']
asia_exchanges = ['SHH', 'JPX', 'HKG']

selected_exchanges = us_exchanges + eu_exchanges + asia_exchanges

full_selected_stocks = {}
df_all_stocks = pd.DataFrame()
for exchange in selected_exchanges:
    print(f'Extracting from {exchange}')
    exchanges = [exchange]
    selected_stocks_dict, ticker_list = get_tickers_stocks(50000, exchanges, 50)

    full_selected_stocks.update(selected_stocks_dict)

    if len(ticker_list) > 0: 
        print('YES')
        df = get_close_prices(ticker_list, period = 2, start = '2022-01-01')
        df_all_stocks = pd.concat([df_all_stocks, df], axis=1)

doubly_listed_tickers = double_listed_stocks(full_selected_stocks)

for ticker_to_drop in doubly_listed_tickers:
    try:
        df_all_stocks = df_all_stocks.drop(columns=[ticker_to_drop])
    except:
        pass


df_all_stocks = df_all_stocks.ffill() #ffill again after concatenating the tickers

Extracting from NMS
YES
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  50 of 50 completed

1 Failed download:
['KSPI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1641013200, endDate = 1704085200")')


Extracting from NYQ
YES


[*********************100%***********************]  50 of 50 completed


Extracting from NGM
YES


...

Extracting from JPX
YES


[*********************100%***********************]  50 of 50 completed

1 Failed download:
['6701.T']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2024-01-01)')
[**                     4%                       ]  2 of 50 completed

Extracting from HKG
YES


[*********************100%***********************]  50 of 50 completed

2 Failed downloads:
['0300.HK', '6936.HK']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1640966400, endDate = 1704038400")')


There is no name found in dict for SOUNW


In [3]:
#GETTING CRYPTOS FROM COINBASE 50 INDEX
#https://www.marketvector.com/factsheets/download/COIN50.d.pdf

coinbase_50_cryptos = ['BTC', 'ETH', 'XRP', 'SOL', 'DOGE', 'ADA', 'LINK', 'XLM', 'AVAX', 'SHIB', 'DOT', 'LTC', 'BCH', 
                       'UNI', 'NEAR', 'PEPE', 'APT', 'ICP', 'ETC', 'AAVE', 'RNDR', 'ATOM', 'MATIC', 'ALGO', 'EOS', 'MKR', 
                       'ASI', 'QNT', 'BONK', 'STX', 'INJ', 'GRT', 'LDO', 'XTZ', 'CRV', 'SAND', 'ZEC', 'HNT', 'JASMY', 'MANA', 
                       'AXS', 'WIF', 'CHZ', 'COMP', 'APE', 'AERO', '1INCH', 'SNX', 'ROSE', 'LPT']

crypto_tickers_fixed = [tick + "-USD" for tick in coinbase_50_cryptos]

cryptos_df = get_close_prices(crypto_tickers_fixed, period = 2, start = '2022-01-01')

print('Number of NA values in cryptos_df is ', df.isna().any().sum())

#cryptos_df

[*********************100%***********************]  50 of 50 completed

1 Failed download:
['ASI-USD']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1640995200, endDate = 1704067200")')


Number of NA values in cryptos_df is  0


In [4]:
#Random Portfolios generation

tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

In [5]:
#Select top five sharpe ratio portfolios from a portfolio

sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)

top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [35]:
df_pct_change = df_all_stocks.pct_change().dropna()
cov_matrix = df_pct_change.cov().values

In [2]:
from pyportfolioopt import EfficientFrontier
from pyportfolioopt import risk_models
from pyportfolioopt import expected_returns

# Assuming df_pct_change is already defined, containing the percentage change in stock prices.

# Step 1: Calculate expected returns and the covariance matrix
mu = expected_returns.mean_historical_return(df_pct_change)  # Expected returns
S = risk_models.sample_cov(df_pct_change)  # Covariance matrix

# Step 2: Create an Efficient Frontier object
ef = EfficientFrontier(mu, S)

# Step 3: Find the minimum variance portfolio
weights = ef.min_volatility()

# Step 4: Print the optimal weights for the minimum variance portfolio
print("Optimal portfolio weights (min variance):")
for stock, weight in weights.items():
    print(f"{stock}: {weight:.4f}")

ModuleNotFoundError: No module named 'pyportfolioopt'

In [1]:
from pyportfolioopt import EfficientFrontier

ModuleNotFoundError: No module named 'pyportfolioopt'